# step_1: feature extractions

In [ ]:
import numpy as np
import collections
import SimpleITK as sitk
from scipy.ndimage.interpolation import zoom
import os,sys
import pandas as pd

from radiomics import featureextractor

In [ ]:
!pwd

In [ ]:
## Set GPU
#os.environ["CUDA_VISIBLE_DEVICES"]="0"

# Load batch file
imgDir = '/home/peng/00_github/03_radiomics/240602_ESCC_ML_SP/example/images'
dirlist = os.listdir(imgDir)[1:]
print(dirlist)


In [ ]:
# 1. define functions to read images in Nifti format 

# read images in Nifti format 
def loadSegArraywithID(fold,iden):
    
    path = fold
    pathList = os.listdir(path)
    
    segPath = [os.path.join(path,i) for i in pathList if ('seg3' in i.lower()) & (iden in i.lower())][0]
    seg = sitk.ReadImage(segPath)
    return seg
# read regions of interest (ROI) in Nifti format 
def loadImgArraywithID(fold,iden):
    
    path = fold
    pathList = os.listdir(path)
    
    imgPath = [os.path.join(path,i) for i in pathList if ('im' in i.lower()) & (iden in i.lower())][0]
    img = sitk.ReadImage(imgPath)    
    return img

# 2. Check the images and ROIs

# 3. Feature Extraction
featureDict = {}
for ind in range(len(dirlist)):
    path = os.path.join(imgDir,dirlist[ind])
    
    # you can make your own pipeline to import data, but it must be SimpleITK images
    mask = loadSegArraywithID(path,'seg3')
    img = loadImgArraywithID(path,'i')
    params = '/home/peng/00_github/03_radiomics/240602_ESCC_ML_SP/Feature Extraction/Paramsescc.yaml'
    
    extractor =  featureextractor.RadiomicsFeatureExtractor(params)
   

    result = extractor.execute(img,mask)
    key = list(result.keys())
    key = key[1:] 
    
    feature = []
    for jind in range(len(key)):
        feature.append(result[key[jind]])
        
    featureDict[dirlist[ind]] = feature
    dictkey = key
    print(dirlist[ind])
    
dataframe = pd.DataFrame.from_dict(featureDict, orient='index', columns=dictkey)
dataframe.to_csv('/home/peng/00_github/03_radiomics/240602_ESCC_ML_SP/example/results/Features_Radiomics.csv')

# step_2: invalid feature processing

In [ ]:
# 1. mean imputation
df.fillna(method=a.mean(),inplace=True)

# 2. nn imputation
#使用每列缺失值前面的值进行填充(按相应列填充,按照相应index前后填充)
df.fillna(method='ffill',inplace=True)
df.fillna(method='pad',inplace=True)
#使用每列缺失值后面的值进行填充(按相应列填充,按照相应index前后填充)
df.fillna(method='backfill',inplace=True)
df.fillna(method='bfill',inplace=True)
#使用每列缺失值前面的值进行填充(按相邻列填充,按照相应columns前后填充)
df.fillna(method='ffill',inplace=True,axis=1)

#上下两个值的平均值进行填充
df=df.fillna(df.interpolate())
#在此详细介绍下interpolate函数的参数：'nearest', 'zero', 'slinear', 'quadratic', 'cubic', 'spline', 'barycentric', 'polynomial'
